In [1]:
!rm -r Image-Captioning/

rm: cannot remove 'Image-Captioning/': No such file or directory


In [2]:
# get the code for kaggle
!git clone https://github.com/moaaztaha/Image-Captioning
py_files_path = '/kaggle/working/Image-Captioning/'
import sys
sys.path.append(py_files_path)

Cloning into 'Image-Captioning'...
remote: Enumerating objects: 431, done.
remote: Counting objects: 100% (431/431), done.
remote: Compressing objects: 100% (166/166), done.
remote: Total 431 (delta 272), reused 421 (delta 262), pack-reused 0
Receiving objects: 100% (431/431), 36.33 MiB | 20.78 MiB/s, done.
Resolving deltas: 100% (272/272), done.


In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
import time 
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence
from models import Encoder, DecoderWithAttention
from dataset import *
from utils import *
from train import *
from nltk.translate.bleu_score import corpus_bleu

In [5]:
# Model parameters
encoder_dim = 2048 # resnet101
emb_dim = 512  # dimension of word embeddings
attention_dim = 512  # dimension of attention linear layers
decoder_dim = 512  # dimension of decoder RNN
dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # sets device for model and PyTorch tensors
cudnn.benchmark = True  # set to true only if inputs to model are fixed size; otherwise lot of computational overhead

# training parameters
epochs = 10  # number of epochs to train for (if early stopping is not triggered)
batch_size = 256
workers = 4
encoder_lr = 1e-4  # learning rate for encoder if fine-tuning
decoder_lr = 4e-4  # learning rate for decoder
fine_tune_encoder = False  # fine-tune encoder?
checkpoint = None  # path to checkpoint, None if none

In [6]:
DATA_NAME = 'flickr30k_5_cap_per_img_2_min_word_freq_resnet101'

# local
# DATA_JSON_PATH = 'data.json'
# IMGS_PATH = 'flickr/Images/'
# kaggle paths
DATA_JSON_PATH = '/kaggle/working/Image-Captioning/data30.json'
IMGS_PATH = '../input/flickr30k/Images/'

In [7]:
# load vocab
vocab = build_vocab(DATA_JSON_PATH)

100%|██████████| 155070/155070 [00:00<00:00, 209062.88it/s]


In [8]:
len(vocab)

12096

In [9]:
t_params = {
    'data_name': DATA_NAME,
    'imgs_path': IMGS_PATH,
    'df_path': DATA_JSON_PATH,
    'vocab': vocab,
    'epochs': epochs,
    'batch_size': batch_size,
    'workers': workers,
    'decoder_lr': decoder_lr,
    'encoder_lr': encoder_lr,
    'fine_tune_encoder': fine_tune_encoder
}

m_params = {
    'attention_dim': attention_dim,
    'embed_dim': emb_dim,
    'decoder_dim': decoder_dim,
    'encoder_dim': encoder_dim,
    'dropout': dropout
}

In [10]:
t_params

{'data_name': 'flickr30k_5_cap_per_img_2_min_word_freq_resnet101',
 'imgs_path': '../input/flickr30k/Images/',
 'df_path': '/kaggle/working/Image-Captioning/data30.json',
 'vocab': <dataset.Vocabulary at 0x7f25030f4610>,
 'epochs': 10,
 'batch_size': 256,
 'workers': 4,
 'decoder_lr': 0.0004,
 'encoder_lr': 0.0001,
 'fine_tune_encoder': False}

In [11]:
fit(t_params=t_params, m_params=m_params)

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth


  0%|          | 0.00/170M [00:00<?, ?B/s]

Loading Data
Dataset split: train
Unique images: 29000
Total size: 145000
Dataset split: val
Unique images: 1014
Total size: 5070
__________________________________________________
-------------------- Fitting --------------------
__________________________________________________
-------------------- Training --------------------
Epoch: [0][0/567]	Batch Time 13.302 (13.302)	Data Load Time 7.781 (7.781)	Loss 10.2997 (10.2997)	Top-5 Accuracy 0.000 (0.000)
Epoch: [0][100/567]	Batch Time 2.662 (2.319)	Data Load Time 0.005 (0.079)	Loss 5.4290 (5.9783)	Top-5 Accuracy 47.560 (40.683)
Epoch: [0][200/567]	Batch Time 2.159 (2.247)	Data Load Time 0.001 (0.040)	Loss 5.0931 (5.5992)	Top-5 Accuracy 51.245 (45.601)
Epoch: [0][300/567]	Batch Time 2.041 (2.229)	Data Load Time 0.001 (0.027)	Loss 5.0117 (5.3999)	Top-5 Accuracy 53.064 (48.124)
Epoch: [0][400/567]	Batch Time 2.014 (2.223)	Data Load Time 0.001 (0.021)	Loss 4.8330 (5.2611)	Top-5 Accuracy 54.742 (49.857)
Epoch: [0][500/567]	Batch Time 2.194 

In [32]:
# Model parameters
encoder_dim = 2048 # resnet101
emb_dim = 512  # dimension of word embeddings
attention_dim = 512  # dimension of attention linear layers
decoder_dim = 512  # dimension of decoder RNN
dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # sets device for model and PyTorch tensors
cudnn.benchmark = True  # set to true only if inputs to model are fixed size; otherwise lot of computational overhead

# training parameters
epochs = 15  # number of epochs to train for (if early stopping is not triggered)
batch_size = 32
workers = 4
encoder_lr = 1e-4  # learning rate for encoder if fine-tuning
decoder_lr = 4e-4  # learning rate for decoder
fine_tune_encoder = True  # fine-tune encoder?
checkpoint = './BEST_checkpoint_flickr30k_5_cap_per_img_2_min_word_freq_resnet101.pth.tar'  # path to checkpoint, None if none

In [33]:
t_params = {
    'data_name': DATA_NAME,
    'imgs_path': IMGS_PATH,
    'df_path': DATA_JSON_PATH,
    'vocab': vocab,
    'epochs': epochs,
    'batch_size': batch_size,
    'workers': workers,
    'decoder_lr': decoder_lr,
    'encoder_lr': encoder_lr,
    'fine_tune_encoder': fine_tune_encoder
}

m_params = {
    'attention_dim': attention_dim,
    'embed_dim': emb_dim,
    'decoder_dim': decoder_dim,
    'encoder_dim': encoder_dim,
    'dropout': dropout
}

t_params

{'data_name': 'flickr30k_5_cap_per_img_2_min_word_freq_resnet101',
 'imgs_path': '../input/flickr30k/Images/',
 'df_path': '/kaggle/working/Image-Captioning/data30.json',
 'vocab': <dataset.Vocabulary at 0x7f25030f4610>,
 'epochs': 15,
 'batch_size': 32,
 'workers': 4,
 'decoder_lr': 0.0004,
 'encoder_lr': 0.0001,
 'fine_tune_encoder': True}

In [34]:
fit(t_params=t_params, checkpoint=checkpoint, m_params=m_params)

Loaded Checkpoint!!
Starting Epoch: 8
Loading Data
Dataset split: train
Unique images: 29000
Total size: 145000
Dataset split: val
Unique images: 1014
Total size: 5070
__________________________________________________
-------------------- Fitting --------------------
__________________________________________________
-------------------- Training --------------------
Epoch: [8][0/4532]	Batch Time 3.173 (3.173)	Data Load Time 1.621 (1.621)	Loss 3.8533 (3.8533)	Top-5 Accuracy 67.382 (67.382)
Epoch: [8][100/4532]	Batch Time 0.438 (0.478)	Data Load Time 0.000 (0.016)	Loss 3.7603 (3.9638)	Top-5 Accuracy 67.797 (65.307)
Epoch: [8][200/4532]	Batch Time 0.439 (0.466)	Data Load Time 0.000 (0.008)	Loss 3.7139 (3.9672)	Top-5 Accuracy 68.610 (65.439)
Epoch: [8][300/4532]	Batch Time 0.424 (0.463)	Data Load Time 0.000 (0.006)	Loss 4.0706 (3.9480)	Top-5 Accuracy 65.337 (65.677)
Epoch: [8][400/4532]	Batch Time 0.443 (0.460)	Data Load Time 0.000 (0.004)	Loss 3.9764 (3.9413)	Top-5 Accuracy 63.007 (65.8